# BS Optimization

- This script will take a fake BS and perform an optimization to determine highest profit generating balances
- After the first successful run, I will layer on additional complexity in terms of contraints

In [178]:
import pandas as pd
import numpy as np
import scipy.optimize as spo

In [179]:
# Define objective function
# Only used in the version of that uses the 'minimize' function

# def fun(x):
    
#     sva = x * nsi
    
#     return -sva

In [180]:
# Define starting balances
# Not need for linprog implementation

# x0 = np.array(df['start'])

# x0.reshape(-1, 1)

In [224]:
# Read in BS data

df = pd.read_excel('./data/sample_bs.xlsx', index_col=0, nrows=38, usecols='A:Z')

In [226]:
# Read in constraints

constraints = pd.read_excel('./data/sample_bs.xlsx', index_col=0, sheet_name='constraints')

In [227]:
constraints

,CET1,T1,total_capital,TLAC
SRWA,0.110,0.125,0.145,0.225
ARWA,0.110,0.125,0.145,0.225
leverage,0.075,0.090,NaN,NaN


In [228]:
df.head()

,Product,start,grow,shrink,spread,A_L,b1_leverage,a_rwa,s_rwa,gsib_leverage,...,gsib_payment,gsib_auc,gsib_underwriting,gsib_otc,gsib_trading,gsib_level3,CET1_resource,T1_resource,total_capital_resource,TLAC_resource
Index,,,,,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,1,0.2,1.0,1,...,0,0,0,0.0,0.0,0.0,0,0,0,0
2,subprime_auto,4000,2000,-1000,150,1,1,0.6,1.0,1,...,0,0,0,0.0,0.0,0.0,0,0,0,0
3,mtg_30_fixed,50000,30000,-15000,70,1,1,0.3,0.5,1,...,0,0,0,0.0,0.0,0.0,0,0,0,0
4,mtg_15_fixed,15000,8000,-3000,72,1,1,0.3,0.5,1,...,0,0,0,0.0,0.0,0.0,0,0,0,0
5,mtg_7_fixed,5000,2500,-2000,70,1,1,0.3,0.5,1,...,0,0,0,0.0,0.0,0.0,0,0,0,0


In [229]:
# Define objective function coefficients
# We are 'minimizing' the spread so will make this negative
# Dividing by 10,000 to convert bps to spread (will not affect optimization)

c = -np.array(df['spread'] / 10000)

## Inequality Constraint

- The inequality constraint for our optimization is the the SRWA % * 11% * the balance - equity needs to be greater than or equal to 0.
- Since it is greater than, for purposes of Scipy notation, we need to flip the sign

### Advanced RWA

In [230]:
# Extract Advanced RWA inequality coefficients

aRWA_mins = list(constraints.loc['ARWA', :])

aRWA_cet1 = list(df['a_rwa'] * aRWA_mins[0] - df['CET1_resource'])
aRWA_t1 = list(df['a_rwa'] * aRWA_mins[1] - df['T1_resource'])
aRWA_tc = list(df['a_rwa'] * aRWA_mins[2] - df['total_capital_resource'])
aRWA_tlac = list(df['a_rwa'] * aRWA_mins[3] - df['TLAC_resource'])

# Combine into one list
aRWA_constraints = [aRWA_cet1, aRWA_t1, aRWA_tc, aRWA_tlac]

### Standardized RWA

In [231]:
# Extract RWA inequality coefficients

sRWA_mins = list(constraints.loc['SRWA', :])

sRWA_cet1 = list(df['s_rwa'] * sRWA_mins[0] - df['CET1_resource'])
sRWA_t1 = list(df['s_rwa'] * sRWA_mins[1] - df['T1_resource'])
sRWA_tc = list(df['s_rwa'] * sRWA_mins[2] - df['total_capital_resource'])
sRWA_tlac = list(df['s_rwa'] * sRWA_mins[3] - df['TLAC_resource'])

# Combine into one list

sRWA_constraints = [sRWA_cet1, sRWA_t1, sRWA_tc, sRWA_tlac]

### Leverage

In [232]:
# Extract leverage inequality constraint

lev_mins = list(constraints.loc['leverage',['CET1', 'T1']])

lev_cet1 = list(df['b1_leverage'] * lev_mins[0] - df['CET1_resource'])
lev_t1 = list(df['b1_leverage'] * lev_mins[1] - df['T1_resource'])

# Combine into one list

lev_constraints = [lev_cet1, lev_t1]

In [233]:
# Combine Advanced RWA and Standardized RWA constraints into a single list

A_ineq = aRWA_constraints + sRWA_constraints + lev_constraints

# Define the other side of the inequality equation (we want to be at least at the minimum, so this is 0)
# There are 10 constraints so this gets multiplied accordingly

b_ineq = [0] * 10

In [234]:
A_ineq

[[0.022000000000000002,
  0.066,
  0.033,
  0.033,
  0.033,
  0.033,
  0.033,
  0.12100000000000001,
  0.1045,
  0.0825,
  0.077,
  0.07150000000000001,
  0.066,
  0.044000000000000004,
  0.055,
  0.0385,
  0.055,
  0.022000000000000002,
  0.0055000000000000005,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  0.0,
  0.0,
  0.0],
 [0.025,
  0.075,
  0.0375,
  0.0375,
  0.0375,
  0.0375,
  0.0375,
  0.1375,
  0.11875,
  0.09375,
  0.0875,
  0.08125,
  0.075,
  0.05,
  0.0625,
  0.04375,
  0.0625,
  0.025,
  0.00625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -1.0,
  -1.0,
  0.0,
  0.0],
 [0.028999999999999998,
  0.087,
  0.0435,
  0.0435,
  0.0435,
  0.0435,
  0.0435,
  0.1595,
  0.13774999999999998,
  0.10874999999999999,
  0.10149999999999999,
  0.09425,
  0.087,
  0.057999999999999996,
  0.0725,
  0.050749999999999997,
  0.0725,
  0.028999999999999998,
  0

### GSIB

- Each point of additional GSIB adds a fixed % to the RWA minimum (in the linear interpolation approach)
- Next step is to figure out how to include as part of the inequality constraint

## Equality Constraint

- This constraint says that Assets - Liabilities must equal to 0.
- I've pre-programmed this to an extent by including a assets (+100%) and liability (-100%) weight in the inputs.

In [235]:
# Create Asset and Liability equality constraint.

A_eq = [list(df['A_L'])]

In [236]:
# Equality constraint vector defined below. This is saying that assets must equal liabilities on the B/S

a_l_constraint = 0

b_eq = [a_l_constraint]

In [237]:
# Create upper and lower bounds. This is the grow and shrink amount for each item

df['u_bound'] = df['start'] + df['grow']
df['l_bound'] = df['start'] + df['shrink']

# Create tuple of lower / upper bounds
df['bounds'] = df.apply(lambda row: tuple((row['l_bound'], row['u_bound'])), axis=1)

In [238]:
# Define bounds in format needed for linprog

bounds = list(df['bounds'])

## Optimize B/S

In [254]:
res = spo.linprog(c, A_ub = A_ineq, b_ub = b_ineq, A_eq = A_eq, b_eq = b_eq, bounds = bounds, 
                  method='revised simplex', options = {"disp": True})

Phase Iteration Minimum Slack       Constraint Residual Objective          
1     0         -88475.1875         158450.0            -13799.05           
1     1         -55824.1625         125798.975          -10533.9475         
1     2         -52371.75           122346.5625         -10188.70625        
1     3         -47935.2            117910.0125         -9745.05125         
1     4         -47566.4375         117541.25           -9708.175           
1     5         -42531.7625         112506.575          -9204.7075          
1     6         -40384.6875         110359.5            -9065.147625        
1     7         -37266.75           107241.5625         -8862.4816875       
1     8         -30413.4            100388.2125         -8725.4146875       
1     9         -27189.0            97163.8125          -8686.7218875       
1     10        0.0                 69974.8125          -8360.4538875       
1     11        0.0                 0.0                 -9759.9501375       


## Results

In [240]:
# This shows that a solution was found

res.success

True

In [241]:
# Show the output of the objective function

res.fun

-14487.49080645161

In [242]:
# Compare to starting profitability

start_profitability = -sum(df['start'] * (df['spread'] / 10000))

(res.fun - start_profitability) / start_profitability

# Profitability increased 50%

0.3756994403619418

In [243]:
# Show the slack against each of the constraints.
# As expect, Standardized RWA (last four values) is significantly more binding than Advanced

res.slack

array([ 1.67662798e+04,  1.90525907e+04,  2.21010052e+04,  3.42946633e+04,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.81898940e-12,
        3.57558468e+03,  3.65959677e+02])

In [245]:
# Show ending balances

res.x

array([ 10000.        ,   6000.        ,  80000.        ,  23000.        ,
         7500.        ,  30000.        ,  13500.        ,  80000.        ,
        17500.        ,  15000.        ,  55652.41935484,  28000.        ,
        85000.        , 105000.        ,  75000.        ,  95000.        ,
        34000.        ,  13000.        ,   1500.        , 235000.        ,
        85000.        , 110000.        ,  26000.        ,  12000.        ,
         3500.        ,  75000.        ,  32000.        ,  22000.        ,
        12500.        ,  16000.        ,   9500.        ,   5000.        ,
         2000.        ,   3000.        ,  61674.51612903,   8410.16129032,
        11213.5483871 ,  44854.19354839])

In [246]:
# Append ending balance to our data and compare growth vs. shrink

df['optimal_balance'] = res.x

df['balance_change'] = df['optimal_balance'] - df['start']

In [247]:
balance_results = df[['Product', 'balance_change']]

In [248]:
# Some products are growing, some are shrinking, and business non-op deposits are our marginal product

balance_results

,Product,balance_change
Index,,
1,prime_auto,-5000.000000
2,subprime_auto,2000.000000
3,mtg_30_fixed,30000.000000
4,mtg_15_fixed,8000.000000
5,mtg_7_fixed,2500.000000
6,mtg_15_arm,10000.000000
7,mtg_7_arm,3500.000000
8,consumer_card,20000.000000
9,business_card,2500.000000


In [176]:
df.head()

,Product,start,grow,shrink,spread,A_L,a_rwa,s_rwa,CET1_resource,T1_resource,total_capital_resource,TLAC_resource,u_bound,l_bound,bounds,optimal_balance,balance_change
Index,,,,,,,,,,,,,,,,,
1,prime_auto,15000,8000,-5000,80,1,0.2,1.0,0,0,0,0,23000,10000,"(10000, 23000)",10000.0,-5000.0
2,subprime_auto,4000,2000,-1000,150,1,0.6,1.0,0,0,0,0,6000,3000,"(3000, 6000)",6000.0,2000.0
3,mtg_30_fixed,50000,30000,-15000,70,1,0.3,0.5,0,0,0,0,80000,35000,"(35000, 80000)",80000.0,30000.0
4,mtg_15_fixed,15000,8000,-3000,72,1,0.3,0.5,0,0,0,0,23000,12000,"(12000, 23000)",23000.0,8000.0
5,mtg_7_fixed,5000,2500,-2000,70,1,0.3,0.5,0,0,0,0,7500,3000,"(3000, 7500)",7500.0,2500.0


In [177]:
# Export updated results to Excel

df.to_excel('./results/results.xlsx')